In [1]:
# %matplotlib notebook
import pandas as pd
import numpy as np
import multiprocessing
import argparse
from datetime import timedelta
import pyvips
from PIL import Image
import time
import random
import gc
import logging
import os
from pathlib import Path
import sys
import matplotlib.pyplot as plt

sys.path.append('..')
from src.wsi import slides, filters, tiles, util
from src import dataset

BASE_PATH = Path(os.path.dirname(os.getcwd()))
DATA_PATH = BASE_PATH / 'data' / 'originals'
PROCESSED_DATA = BASE_PATH / 'data' / 'processed'

In [2]:
low = pd.read_csv(BASE_PATH / 'data' / 'lowres.csv')
hi = pd.read_csv(BASE_PATH / 'data' / 'hires.csv')
# row = train.iloc[2]
# image = pyvips.Image.new_from_file(f'../data/originals/{row["filename"]}', access='sequential')
# image.filename, image.width, image.height

In [3]:
names = !ls ../workspace/tile_data/48
names = [x.split('-')[0] for x in names]

In [13]:
from glob import glob
set1 = set(hi['filename'].str.split('.').str[0])
set2 = set(names)
dels = set2.intersection(set1)
len(dels)
for file in dels:
    f = f'../workspace/tile_data/48/{file}-tile_data.csv'
    os.remove(f)
    f = f'../workspace/tile_data/64/{file}-tile_data.csv'
    os.remove(f)
    f = f'../workspace/tiles/data/processed/*/*/{file}*'
    files = glob(f)
    for fil in files:
        os.remove(fil)

In [3]:
image.get('bands'), image.get('coding'), image.get('format'), image.get('interpretation')

(3, 'none', 'uchar', 'srgb')

In [8]:
image.tiffsave('test.tif', compression="deflate",
                 tile=True, tile_width=256, tile_height=256,
                 pyramid=True)

In [56]:
def rimages(index, train, page):
    filename, res = train.loc[index, ['filename', 'resolution']]
    resizing_factor = res / 0.25
    image = pyvips.Image.new_from_file(f'../data/originals/{filename}', page=page)
    width, height = image.width, image.height
    image = image.affine((resizing_factor, 0, 0, resizing_factor))
    return width, height, image, image.width, image.height

In [3]:
images = pyvips.Image.new_from_file('../data/originals/qmsqv731.tif', page=8)
images.height, images.width
img = images.write_to_file('test.jpg')
# img = np.ndarray(buffer=images.write_to_memory(), dtype=np.uint8, shape=[images.height, images.width, images.bands])
plt.imshow(plt.imread('test.jpg'))


In [10]:
plt.imshow(images.numpy() / 255.)
plt.show()

In [3]:
img = Image.open('test.jpg')
plt.imshow(img)

In [8]:
train.iloc[:, :4].to_csv('../data/train.csv', index=False)

In [9]:
dftrain = dataset.get_dftrain('../workspace/tiles/data/processed/48/3/', 5, 42, 0)
dftrain

training dataset: 1342 samples
training dataset: 126581 tiles


,tissue_id,filename,relapse,split,n_tiles,path,tile_id,is_valid,age,sex,melanoma_history
0,00shcdng,00shcdng.tif,0,3,96,../workspace/tiles/data/processed/48/3/00shcdng_0.jpeg,0,0,0.795918,0,-1
1,00shcdng,00shcdng.tif,0,3,96,../workspace/tiles/data/processed/48/3/00shcdng_1.jpeg,1,0,0.795918,0,-1
2,00shcdng,00shcdng.tif,0,3,96,../workspace/tiles/data/processed/48/3/00shcdng_2.jpeg,2,0,0.795918,0,-1
3,00shcdng,00shcdng.tif,0,3,96,../workspace/tiles/data/processed/48/3/00shcdng_3.jpeg,3,0,0.795918,0,-1
4,00shcdng,00shcdng.tif,0,3,96,../workspace/tiles/data/processed/48/3/00shcdng_4.jpeg,4,0,0.795918,0,-1
...,...,...,...,...,...,...,...,...,...,...,...
126576,zyo5fbtd,zyo5fbtd.tif,1,2,35,../workspace/tiles/data/processed/48/3/zyo5fbtd_30.jpeg,30,0,0.653061,0,-1
126577,zyo5fbtd,zyo5fbtd.tif,1,2,35,../workspace/tiles/data/processed/48/3/zyo5fbtd_31.jpeg,31,0,0.653061,0,-1
126578,zyo5fbtd,zyo5fbtd.tif,1,2,35,../workspace/tiles/data/processed/48/3/zyo5fbtd_32.jpeg,32,0,0.653061,0,-1
126579,zyo5fbtd,zyo5fbtd.tif,1,2,35,../workspace/tiles/data/processed/48/3/zyo5fbtd_33.jpeg,33,0,0.653061,0,-1


In [10]:
dls = dataset.get_dataloader(dftrain)
#tr, val = dblock.splitter(dblock.get_items(dftrain))
#dataset.show_tile_batch(dls)

In [46]:
tr = dls.train

In [4]:
%matplotlib inline

In [12]:
x, y = dls.valid.one_batch()
x.shape

torch.Size([1, 48, 3, 512, 512])